In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

file_path = 'D:\practice\solar_weather.csv\solar_weather.csv'
data = pd.read_csv(file_path)

data['Time'] = pd.to_datetime(data['Time'])
data['year'] = data['Time'].dt.year
yearly_data = data.groupby('year').agg({
    'Energy delta[Wh]': 'sum',
    'GHI': 'mean',
    'temp': 'mean',
    'pressure': 'mean',
    'humidity': 'mean',
    'wind_speed': 'mean'
}).reset_index()

# Dash app
app = dash.Dash(__name__)

app.layout = html.Div(style={'backgroundColor': '#1f1f1f', 'color': 'white', 'font-family': 'Arial'}, children=[
    html.H1('Solar and Weather Data Dashboard', style={'textAlign': 'center', 'color': '#00FF00'}),

    dcc.Tabs(id='tabs', value='trends', children=[
        dcc.Tab(label='Trends', value='trends', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'}),
        dcc.Tab(label='Correlation', value='correlation', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'}),
        dcc.Tab(label='Monthly Energy', value='monthly-energy', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'}),
        dcc.Tab(label='Hourly Patterns', value='hourly-patterns', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'}),
        dcc.Tab(label='Temperature', value='temperature', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'}),
        dcc.Tab(label='GHI', value='ghi', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'}),
        dcc.Tab(label='Wind Speed', value='wind-speed', style={'backgroundColor': '#333', 'borderRadius': '10px', 'color': 'white', 'padding': '10px', 'margin': '5px'}, selected_style={'backgroundColor': '#00FF00', 'borderRadius': '10px', 'color': 'black'})
    ], style={'backgroundColor': '#444', 'borderRadius': '10px', 'color': 'white'}),
    
    html.Div(id='tabs-content', style={'padding': '20px'})
])

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'trends':
        fig_energy = go.Figure(data=[go.Scatter(x=yearly_data['year'], y=yearly_data['Energy delta[Wh]'], mode='lines+markers', line=dict(color='lime'), name='Energy Production')])
        fig_temp = go.Figure(data=[go.Scatter(x=yearly_data['year'], y=yearly_data['temp'], mode='lines+markers', line=dict(color='lime'), name='Average Temperature')])
        
        return html.Div([
            html.H2('Trends Over the Years', style={'textAlign': 'center', 'color': '#00FF00'}),
            html.Div([
                html.Div([
                    html.H3('Energy Production by Year', style={'textAlign': 'center', 'color': '#00FF00'}),
                    dcc.Graph(figure=fig_energy)
                ], style={'width': '48%', 'display': 'inline-block'}),
                html.Div([
                    html.H3('Average Temperature by Year', style={'textAlign': 'center', 'color': '#00FF00'}),
                    dcc.Graph(figure=fig_temp)
                ], style={'width': '48%', 'display': 'inline-block'})
            ], style={'display': 'flex', 'justifyContent': 'space-around'})
        ])
    elif tab == 'correlation':
        return html.Div([
            html.H2('Correlation Matrix', style={'textAlign': 'center', 'color': '#00FF00'}),
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Heatmap(z=correlation.values, x=correlation.columns, y=correlation.columns, colorscale='greens')],
                    layout=go.Layout(
                        title='Correlation Matrix',
                        xaxis=dict(tickfont=dict(color='white')),
                        yaxis=dict(tickfont=dict(color='white')),
                        plot_bgcolor='black', 
                        paper_bgcolor='black'
                    )
                )
            )
        ], style={'margin': '20px'})
    elif tab == 'monthly-energy':
        monthly_data = data.groupby('month').agg({'Energy delta[Wh]': 'sum'}).reset_index()
        return html.Div([
            html.H2('Monthly Energy Production', style={'textAlign': 'center', 'color': '#00FF00'}),
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Bar(x=monthly_data['month'], y=monthly_data['Energy delta[Wh]'], marker=dict(color='green'))],
                    layout=go.Layout(title='Monthly Energy Production', plot_bgcolor='black', paper_bgcolor='black', xaxis=dict(tickfont=dict(color='white')), yaxis=dict(tickfont=dict(color='white')))
                )
            )
        ], style={'margin': '20px'})
    elif tab == 'hourly-patterns':
        hourly_data = data.groupby('hour').agg({'Energy delta[Wh]': 'sum'}).reset_index()
        return html.Div([
            html.H2('Hourly Energy Production', style={'textAlign': 'center', 'color': '#00FF00'}),
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Scatter(x=hourly_data['hour'], y=hourly_data['Energy delta[Wh]'], mode='lines+markers', line=dict(color='green'))],
                    layout=go.Layout(title='Hourly Energy Production', plot_bgcolor='black', paper_bgcolor='black', xaxis=dict(tickfont=dict(color='white')), yaxis=dict(tickfont=dict(color='white')))
                )
            )
        ], style={'margin': '20px'})
    elif tab == 'temperature':
        return html.Div([
            html.H2('Temperature Over Time', style={'textAlign': 'center', 'color': '#00FF00'}),
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Scatter(x=data['Time'], y=data['temp'], mode='lines', line=dict(color='green'))],
                    layout=go.Layout(title='Temperature Over Time', plot_bgcolor='black', paper_bgcolor='black', xaxis=dict(tickfont=dict(color='white')), yaxis=dict(tickfont=dict(color='white')))
                )
            )
        ], style={'margin': '20px'})
    elif tab == 'ghi':
        return html.Div([
            html.H2('Global Horizontal Irradiance (GHI) Over Time', style={'textAlign': 'center', 'color': '#00FF00'}),
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Scatter(x=data['Time'], y=data['GHI'], mode='lines', line=dict(color='green'))],
                    layout=go.Layout(title='GHI Over Time', plot_bgcolor='black', paper_bgcolor='black', xaxis=dict(tickfont=dict(color='white')), yaxis=dict(tickfont=dict(color='white')))
                )
            )
        ], style={'margin': '20px'})
    elif tab == 'wind-speed':
        return html.Div([
            html.H2('Wind Speed Distribution', style={'textAlign': 'center', 'color': '#00FF00'}),
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Histogram(x=data['wind_speed'], marker=dict(color='green'))],
                    layout=go.Layout(title='Wind Speed Distribution', plot_bgcolor='black', paper_bgcolor='black', xaxis=dict(tickfont=dict(color='white')), yaxis=dict(tickfont=dict(color='white')))
                )
            )
        ], style={'margin': '20px'})

if __name__ == '__main__':
    app.run_server(debug=True)
